In [1]:
# Import dependencies
import pandas as pd
from sqlalchemy import create_engine
from scipy import stats

In [2]:
# Import our mental health dataset and create a dataframe
mental_df = pd.read_csv('./Resources/mental-heath-in-tech-2016_20161114.csv')
mental_df.head()

,Are you self-employed?,How many employees does your company or organization have?,Is your employer primarily a tech company/organization?,Is your primary role within your company related to tech/IT?,Does your employer provide mental health benefits as part of healthcare coverage?,Do you know the options for mental health care available under your employer-provided coverage?,"Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?",Does your employer offer resources to learn more about mental health concerns and options for seeking help?,Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?,"If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:",...,"If you have a mental health issue, do you feel that it interferes with your work when being treated effectively?","If you have a mental health issue, do you feel that it interferes with your work when NOT being treated effectively?",What is your age?,What is your gender?,What country do you live in?,What US state or territory do you live in?,What country do you work in?,What US state or territory do you work in?,Which of the following best describes your work position?,Do you work remotely?
0,0,26-100,1.0,NaN,Not eligible for coverage / N/A,NaN,No,No,I don't know,Very easy,...,Not applicable to me,Not applicable to me,39,Male,United Kingdom,NaN,United Kingdom,NaN,Back-end Developer,Sometimes
1,0,6-25,1.0,NaN,No,Yes,Yes,Yes,Yes,Somewhat easy,...,Rarely,Sometimes,29,male,United States of America,Illinois,United States of America,Illinois,Back-end Developer|Front-end Developer,Never
2,0,6-25,1.0,NaN,No,NaN,No,No,I don't know,Neither easy nor difficult,...,Not applicable to me,Not applicable to me,38,Male,United Kingdom,NaN,United Kingdom,NaN,Back-end Developer,Always
3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Sometimes,Sometimes,43,male,United Kingdom,NaN,United Kingdom,NaN,Supervisor/Team Lead,Sometimes
4,0,6-25,0.0,1.0,Yes,Yes,No,No,No,Neither easy nor difficult,...,Sometimes,Sometimes,43,Female,United States of America,Illinois,United States of America,Illinois,Executive Leadership|Supervisor/Team Lead|Dev ...,Sometimes


In [3]:
# check the number of rows and columns in the dataframe
mental_df.shape

(1433, 63)

In [4]:
# Check for low response rate columns < 70% response rate
low_response_columns = []
for column in mental_df.columns:
    if ((mental_df[column].count() / 1433) < .7):
        low_response_columns.append(column)

# display a list of low response columns
low_response_columns

['Is your primary role within your company related to tech/IT?',
 'Do you have medical coverage (private insurance or state-provided) which includes treatment of \xa0mental health issues?',
 'Do you know local or online resources to seek help for a mental health disorder?',
 'If you have been diagnosed or treated for a mental health disorder, do you ever reveal this to clients or business contacts?',
 'If you have revealed a mental health issue to a client or business contact, do you believe this has impacted you negatively?',
 'If you have been diagnosed or treated for a mental health disorder, do you ever reveal this to coworkers or employees?',
 'If you have revealed a mental health issue to a coworker or employee, do you believe this has impacted you negatively?',
 'Do you believe your productivity is ever affected by a mental health issue?',
 'If yes, what percentage of your work time (time performing primary or secondary job functions) is affected by a mental health issue?',
 'Ha

In [5]:
# drop low response columns
mental_df.drop(columns=low_response_columns, inplace=True)

# check the new size of the dataframe
mental_df.shape

(1433, 48)

In [6]:
# Check for many distinct answers indicating long for response or irrelavant data
many_distinct_columns = []
for column in mental_df.columns:
    if (mental_df[column].nunique() > 10):
        many_distinct_columns.append(column)
    

many_distinct_columns.remove('What is your age?')
many_distinct_columns.remove('What country do you live in?')
many_distinct_columns.remove('What country do you work in?')
many_distinct_columns.remove('What is your gender?')



many_distinct_columns

['Why or why not?',
 'Why or why not?.1',
 'Which of the following best describes your work position?']

In [7]:
# drop many distinct answers columns
mental_df.drop(columns=many_distinct_columns, inplace=True)

# see the new size of the dataframe
mental_df.shape

(1433, 45)

In [8]:
# Fix gender data to be consistent
mental_df['What is your gender?'] = mental_df['What is your gender?'].replace('male', 'Male')
mental_df['What is your gender?'] = mental_df['What is your gender?'].replace('m', 'Male')
mental_df['What is your gender?'] = mental_df['What is your gender?'].replace('M', 'Male')
mental_df['What is your gender?'] = mental_df['What is your gender?'].replace('f', 'Male')
mental_df['What is your gender?'] = mental_df['What is your gender?'].replace('F', 'Female')
mental_df['What is your gender?'] = mental_df['What is your gender?'].replace('female', 'Female')

In [9]:
# Print out gender counts
gender_counts = mental_df['What is your gender?'].value_counts()
gender_counts

Male                                                                                                                                                             1047
Female                                                                                                                                                            286
Male                                                                                                                                                               11
Female                                                                                                                                                              9
non-binary                                                                                                                                                          4
                                                                                                                                                                 ... 
Cis 

In [10]:
# Determine which values to replace
replace_genders = list(gender_counts[gender_counts < 20].index)

# Replace in DataFrame
for gender in replace_genders:
    mental_df['What is your gender?'] = mental_df['What is your gender?'].replace(gender,"Other")


In [11]:
# Print out gender counts
gender_counts = mental_df['What is your gender?'].value_counts()
gender_counts

Male      1047
Female     286
Other       97
Name: What is your gender?, dtype: int64

In [12]:
# Print out country counts
country_counts = mental_df['What country do you live in?'].value_counts()
country_counts

United States of America    840
United Kingdom              180
Canada                       78
Germany                      58
Netherlands                  48
Australia                    35
Sweden                       19
France                       16
Ireland                      15
Brazil                       10
Switzerland                  10
India                         9
New Zealand                   9
Russia                        9
Finland                       7
Bulgaria                      7
Denmark                       7
Belgium                       5
Italy                         5
Spain                         4
Romania                       4
Poland                        4
Austria                       4
South Africa                  4
Norway                        3
Chile                         3
Czech Republic                3
Pakistan                      3
Israel                        2
Afghanistan                   2
Colombia                      2
Mexico  

In [13]:
# Determine which values to replace
replace_countries = list(country_counts[country_counts < 100].index)

# Replace in DataFrame
for country in replace_countries:
    mental_df['What country do you live in?'] = mental_df['What country do you live in?'].replace(country,"Other")

In [14]:
# Print out country counts
country_counts = mental_df['What country do you live in?'].value_counts()
country_counts

United States of America    840
Other                       413
United Kingdom              180
Name: What country do you live in?, dtype: int64

In [15]:
# Print out country counts
country_counts = mental_df['What country do you work in?'].value_counts()
country_counts

United States of America    851
United Kingdom              183
Canada                       74
Germany                      58
Netherlands                  47
Australia                    34
Sweden                       20
Ireland                      15
France                       14
Brazil                       10
Switzerland                  10
Russia                        9
New Zealand                   9
India                         9
Finland                       7
Bulgaria                      7
Denmark                       7
Belgium                       5
Poland                        4
Austria                       4
South Africa                  4
Chile                         3
Romania                       3
Norway                        3
Spain                         3
Czech Republic                3
Italy                         3
Israel                        2
Afghanistan                   2
Colombia                      2
Mexico                        2
Other   

In [16]:
# Determine which values to replace
replace_countries = list(country_counts[country_counts < 100].index)

# Replace in DataFrame
for country in replace_countries:
    mental_df['What country do you work in?'] = mental_df['What country do you work in?'].replace(country,"Other")

In [17]:
# Print out country counts
country_counts = mental_df['What country do you work in?'].value_counts()
country_counts

United States of America    851
Other                       399
United Kingdom              183
Name: What country do you work in?, dtype: int64

In [18]:
# function for comparing two columns with a chi-square-test

def chi_square(column):
    crosstab = pd.crosstab(mental_df['Have you been diagnosed with a mental health condition by a medical professional?'], mental_df[column])
    return stats.chi2_contingency(crosstab)[1]

In [19]:
# run chi-square test and drop non significant columns

insignificant_p_columns = []
for column in mental_df.columns:
    p_value = chi_square(column)
    if p_value >= .05:
        insignificant_p_columns.append(column)
        
insignificant_p_columns

['Are you self-employed?',
 'How many employees does your company or organization have?',
 'Is your employer primarily a tech company/organization?',
 'Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?',
 'Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?',
 'Would you feel comfortable discussing a mental health disorder with your coworkers?',
 'Would you feel comfortable discussing a mental health disorder with your direct supervisor(s)?',
 'Did your previous employers provide resources to learn more about mental health issues and how to seek help?',
 'Do you think that discussing a physical health issue with previous employers would have negative consequences?',
 'Would you have been willing to discuss a mental health issue with your previous co-workers?',
 'Would you be willing to bring up a physical health 

In [20]:
# build a dataframe of the questions
dict = {}
count = 0
for column in mental_df.columns:
    
    dict[count] = [column]
    count = count + 1
    
    
dict

questions_df = pd.DataFrame.from_dict(dict, orient='index', columns=['Question'])
questions_df

,Question
0,Are you self-employed?
1,How many employees does your company or organi...
2,Is your employer primarily a tech company/orga...
3,Does your employer provide mental health benef...
4,Do you know the options for mental health care...
5,Has your employer ever formally discussed ment...
6,Does your employer offer resources to learn mo...
7,Is your anonymity protected if you choose to t...
8,If a mental health issue prompted you to reque...
9,Do you think that discussing a mental health d...


In [21]:
# rename the columns in the dataframe to numbers
column_rename = {}
count = 0
for column in mental_df.columns:
    
    column_rename[column] = count
    count = count + 1
    
    
column_rename

renamed_df = mental_df.rename(columns=column_rename)
renamed_df

,0,1,2,3,4,5,6,7,8,9,...,35,36,37,38,39,40,41,42,43,44
0,0,26-100,1.0,Not eligible for coverage / N/A,NaN,No,No,I don't know,Very easy,No,...,No,Yes,0,Not applicable to me,Not applicable to me,39,Male,United Kingdom,United Kingdom,Sometimes
1,0,6-25,1.0,No,Yes,Yes,Yes,Yes,Somewhat easy,No,...,Yes,Yes,1,Rarely,Sometimes,29,Male,United States of America,United States of America,Never
2,0,6-25,1.0,No,NaN,No,No,I don't know,Neither easy nor difficult,Maybe,...,No,No,1,Not applicable to me,Not applicable to me,38,Other,United Kingdom,United Kingdom,Always
3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Yes,Yes,1,Sometimes,Sometimes,43,Male,United Kingdom,United Kingdom,Sometimes
4,0,6-25,0.0,Yes,Yes,No,No,No,Neither easy nor difficult,Yes,...,Yes,Yes,1,Sometimes,Sometimes,43,Female,United States of America,United States of America,Sometimes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1428,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,No,No,1,Not applicable to me,Not applicable to me,34,Other,United States of America,United States of America,Sometimes
1429,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,No,Yes,0,Sometimes,Often,56,Other,United States of America,Other,Sometimes
1430,0,100-500,1.0,Yes,Yes,Yes,Yes,I don't know,Somewhat difficult,Maybe,...,Maybe,Yes,1,Rarely,Sometimes,52,Male,United States of America,United States of America,Sometimes
1431,0,100-500,0.0,I don't know,I am not sure,No,Yes,I don't know,Somewhat difficult,Maybe,...,Yes,Yes,0,Sometimes,Often,30,Female,United States of America,United States of America,Sometimes


In [22]:
# Drop NA values in dataframe
renamed_df.dropna(inplace=True)
renamed_df.shape

(861, 45)

In [23]:
# store data in database
engine = create_engine('sqlite:///mental_health.db', echo=False)
renamed_df.to_sql('pre_encoded_insignificant', con=engine, if_exists='replace')
questions_df.to_sql('pre_encoded_questions_insignificant', con=engine, if_exists='replace')
#engine.execute("SELECT * FROM pre_encoded").fetchall()

In [24]:
mental_df.shape

(1433, 45)

In [25]:
# drop insignificant p value columns
mental_df.drop(columns=insignificant_p_columns, inplace=True)
mental_df.shape

(1433, 32)

In [26]:
# View the dataframe
mental_df

,Does your employer provide mental health benefits as part of healthcare coverage?,Do you know the options for mental health care available under your employer-provided coverage?,Does your employer offer resources to learn more about mental health concerns and options for seeking help?,"If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:",Do you think that discussing a mental health disorder with your employer would have negative consequences?,Do you think that discussing a physical health issue with your employer would have negative consequences?,Do you feel that your employer takes mental health as seriously as physical health?,Have you heard of or observed negative consequences for co-workers who have been open about mental health issues in your workplace?,Do you have previous employers?,Have your previous employers provided mental health benefits?,...,Do you have a family history of mental illness?,Have you had a mental health disorder in the past?,Do you currently have a mental health disorder?,Have you been diagnosed with a mental health condition by a medical professional?,Have you ever sought treatment for a mental health issue from a mental health professional?,"If you have a mental health issue, do you feel that it interferes with your work when being treated effectively?","If you have a mental health issue, do you feel that it interferes with your work when NOT being treated effectively?",What is your gender?,What country do you live in?,What country do you work in?
0,Not eligible for coverage / N/A,NaN,No,Very easy,No,No,I don't know,No,1,"No, none did",...,No,Yes,No,Yes,0,Not applicable to me,Not applicable to me,Male,United Kingdom,United Kingdom
1,No,Yes,Yes,Somewhat easy,No,No,Yes,No,1,"Yes, they all did",...,Yes,Yes,Yes,Yes,1,Rarely,Sometimes,Male,United States of America,United States of America
2,No,NaN,No,Neither easy nor difficult,Maybe,No,I don't know,No,1,"No, none did",...,No,Maybe,No,No,1,Not applicable to me,Not applicable to me,Other,United Kingdom,United Kingdom
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Some did,...,No,Yes,Yes,Yes,1,Sometimes,Sometimes,Male,United Kingdom,United Kingdom
4,Yes,Yes,No,Neither easy nor difficult,Yes,Maybe,No,No,1,I don't know,...,Yes,Yes,Yes,Yes,1,Sometimes,Sometimes,Female,United States of America,United States of America
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1428,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,"Yes, they all did",...,Yes,No,No,No,1,Not applicable to me,Not applicable to me,Other,United States of America,United States of America
1429,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,...,Yes,No,No,Yes,0,Sometimes,Often,Other,United States of America,Other
1430,Yes,Yes,Yes,Somewhat difficult,Maybe,Maybe,I don't know,Yes,1,Some did,...,Yes,Yes,Maybe,Yes,1,Rarely,Sometimes,Male,United States of America,United States of America
1431,I don't know,I am not sure,Yes,Somewhat difficult,Maybe,No,No,No,1,"No, none did",...,Yes,Maybe,Yes,Yes,0,Sometimes,Often,Female,United States of America,United States of America


In [27]:
# Drop NA values in dataframe
mental_df.dropna(inplace=True)

In [28]:
# check the size of the dataframe
mental_df.shape

(861, 32)

In [29]:
# export to tab seperated file
mental_df.to_csv('./Resources/cleaned_data.tsv', sep='\t', index=False)